In [ ]:
from pyomo.environ import *
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from dash import Dash, dcc, html
import webbrowser
from threading import Timer
import dash_bootstrap_components as dbc
import plotly.io as pio
from dash.dependencies import Input, Output
import json
from scripts.result_export import generate_html
from scripts.price_process import price_process


# Read Data

In [ ]:
excel_mode = 2

In [ ]:
if excel_mode == 1:
    # Read price data from Excel
    price_data = pd.read_excel('data/price_data.xlsx', sheet_name= None, index_col=0)

    data = price_data

    # Iteratively merge all dataframes on 'Timestamp'
    price_df = pd.DataFrame(index =  data[list(data.keys())[0]].index)

    for key in data:
        price_df = pd.concat([price_df, data[key]], axis = 1)


    energy_price = pd.read_excel('data/btm/energy_price.xlsx', index_col=0)
    demand_price = pd.read_excel('data/btm/demand_price.xlsx', index_col=0)
    load_profile = pd.read_excel('data/btm/load_profile.xlsx', index_col=0)
    
    model_time_period = 24    # periods
    dt = 1



In [ ]:
data = pd.read_excel('data/full/data.xlsx', sheet_name= None, index_col=0)

In [ ]:
# Parameters
λ_min = data['soc_limit']['min']
λ_max = data['soc_limit']['max']
theta = data['schedule']
beta = data['activation']
gamma = data['reserve']
price_df = data['prices']
load_profile = data['load'].value


z = {}
z['arb'], z['reg'], z['pres'], z['cres'], z['ec'], z['dr'], z['il'], z['symmetric_reg'], z['activate'] = data['basis'].dropna().value.values


In [ ]:
filename = "test"
model_time_period = 48    # periods
dt = 0.5

cap_power = 1  # Example value
cap_energy = 4  # Example value

Seff = 0.9995
Ceff = 0.9
Deff = 0.9

λ_0 = 0.5
λ_max_tech = 1
λ_min_tech = 0.1
initial_soc = λ_0 * cap_energy

cycle_life = 3650
calendar_life = 10
total_time_period = len(price_df)
num_slices = int(total_time_period // model_time_period)     # number of time slices to model

η = cycle_life/calendar_life/365                 # number of cycle per time slice

# Model

In [ ]:
def optimize_revenue(initial_soc, price_vector, current_period, first_period, last_period):

    # Define the model
    model = ConcreteModel()

    # Price Data for the day
    total_time_period = len(price_vector)

    T = range(1, total_time_period + 1)
    SOC_T = range(0, total_time_period + 1)

    p_arb = {t: price_vector['arb_energy_price'].values[t-1] for t in T}

    p_reg_up_c = {t: price_vector['reg_capacity_price'].values[t-1] for t in T}
    p_reg_up_e = {t: price_vector['reg_energy_price'].values[t-1] for t in T}
    p_reg_down_c = {t: price_vector['reg_down_capacity_price'].values[t-1] for t in T}
    p_reg_down_e = {t: price_vector['reg_down_energy_price'].values[t-1] for t in T}

    p_pres_c = {t: price_vector['pres_capacity_price'].values[t-1] for t in T}
    p_pres_e = {t: price_vector['pres_energy_price'].values[t-1] for t in T}
    p_cres_c = {t: price_vector['cres_capacity_price'].values[t-1] for t in T}
    p_cres_e = {t: price_vector['cres_energy_price'].values[t-1] for t in T}

    p_ec = {t: price_vector['ec_energy_price'].values[t-1] for t in T}
    p_dr_c = {t: price_vector['dr_capacity_price'].values[t-1] for t in T}
    p_dr_e = {t: price_vector['dr_energy_price'].values[t-1] for t in T}
    p_il_c = {t: price_vector['il_capacity_price'].values[t-1] for t in T}
    p_il_e = {t: price_vector['il_energy_price'].values[t-1] for t in T}

    start_soc = λ_0 * cap_energy

    # Variables
    model.SOC = Var(SOC_T, within=NonNegativeReals, bounds = (0, cap_energy))
    # model.E_c = Var(T, within=NonNegativeReals, bounds = (0, cap_energy))
    # model.E_cr = Var(T, within=NonNegativeReals, bounds = (0, cap_energy))
    # model.E_d = Var(T, within=NonNegativeReals, bounds = (0, cap_energy))
    # model.E_dr = Var(T, within=NonNegativeReals, bounds = (0, cap_energy))
 
    model.PC_arb = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.PD_arb = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.P_reg = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.PC_reg = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.PD_reg = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.P_pres = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.P_cres = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))

    model.P_ec = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.PC_ec = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.PL_ec = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.PD_L = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.P_dr = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))
    model.P_il = Var(T, within=NonNegativeReals, bounds = (0, cap_power * 2))

    model.y_ch = Var(T, within=Binary)
    model.y_minus = Var(T, within=Binary)

    model.R_arb = Var(T, within=Reals)
    model.R_reg = Var(T, within=Reals)
    model.R_pres = Var(T, within=Reals)
    model.R_cres = Var(T, within=Reals)
    model.R_ec = Var(T, within=Reals)
    model.R_dr = Var(T, within=Reals)
    model.R_il = Var(T, within=Reals)

    # Objective function
    model.obj = Objective(
        expr= sum(model.R_arb[t] + model.R_reg[t] + model.R_pres[t] + model.R_cres[t] 
                  + model.R_ec[t] + model.R_dr[t] + model.R_il[t] 
                  for t in T),        
        sense=maximize
    )

    def revenue_arbitrage_rule(model, t):
        return model.R_arb[t] == p_arb[t]  * (model.PD_arb[t] - model.PC_arb[t]) * dt
    model.revenue_arbitrage = Constraint(T, rule=revenue_arbitrage_rule)

    def revenue_primary_reserve_rule(model, t):
        return model.R_pres[t] == (p_pres_c[t]  * model.P_pres[t]
                                        + p_pres_e[t] * beta['pres'][t] * model.P_pres[t]) * dt
    model.revenue_preserve = Constraint(T, rule=revenue_primary_reserve_rule)

    def revenue_contingency_reserve_rule(model, t):
        return model.R_cres[t] == (p_cres_c[t]  * model.P_cres[t]
                                        + p_cres_e[t] * beta['cres'][t] * model.P_cres[t]) * dt
    model.revenue_creserve = Constraint(T, rule=revenue_contingency_reserve_rule)

    if z['symmetric_reg'] == 1:
        def reg_constraint_rule(model, t):
            return model.P_reg[t] == model.PC_reg[t] + model.PD_reg[t]
        model.reg_up_down = Constraint(T, rule=reg_constraint_rule)

        def revenue_regulation_rule(model, t):
            return model.R_reg[t] == (p_reg_up_c[t]  * model.P_reg[t]
                                            + p_reg_up_e[t] * beta['reg'][t] * model.P_reg[t]) * dt
        model.revenue_regulation = Constraint(T, rule=revenue_regulation_rule)

    else:

        def revenue_regulation_rule(model, t):
            return model.R_reg[t] == (p_reg_up_c[t]  * model.PD_reg[t] + p_reg_down_c[t]  * model.PC_reg[t]
                                            + p_reg_up_e[t] * beta['reg_up'][t] * model.PD_reg[t]
                                            + p_reg_down_e[t] * beta['reg_down'][t] * model.PC_reg[t]
                                            ) * dt
        model.revenue_regulation = Constraint(T, rule=revenue_regulation_rule)


    def revenue_energy_charge_rule(model, t):
        return model.R_ec[t] == p_arb[t]  * (- model.PC_ec[t]) * dt
    model.revenue_energy_charge = Constraint(T, rule=revenue_energy_charge_rule)

    def revenue_demand_response_rule(model, t):
        return model.R_dr[t] == (p_dr_c[t]  * model.P_dr[t]
                                        + p_dr_e[t] * beta['dr'][t] * model.P_dr[t]) * dt
    model.revenue_demand_response = Constraint(T, rule=revenue_demand_response_rule)

    def revenue_interruptible_load_rule(model, t):
        return model.R_il[t] == (p_il_c[t]  * model.P_il[t]
                                        + p_il_e[t] * beta['il'][t] * model.P_il[t]) * dt
    model.revenue_interruptible_load = Constraint(T, rule=revenue_interruptible_load_rule)


    # Power balance
    def total_charge_rule(model, t):
        return model.PC_arb[t] + model.PC_reg[t] + model.PC_ec[t]  <= cap_power * model.y_minus[t]
    model.total_charge = Constraint(T, rule=total_charge_rule)

    def total_discharge_rule(model, t):
        return model.PD_arb[t] +  model.PD_reg[t] + model.P_pres[t] +  model.P_cres[t] \
                    + model.PD_L[t] + model.P_dr[t] +  model.P_il[t] <= cap_power * (1 - model.y_minus[t])
    model.total_discharge = Constraint(T, rule=total_discharge_rule)

    # Power balance for BTM services
    def total_grid_power_rule(model, t):
        return model.P_ec[t] == model.PL_ec[t] + model.PC_ec[t]
    model.total_grid_power = Constraint(T, rule=total_grid_power_rule)

    def total_load_balance_rule(model, t):
        return (model.PL_ec[t] +  model.PD_L[t] + beta['dr'][t] * model.P_dr[t] + beta['il'][t] * model.P_il[t]) * dt == load_profile[t]
    model.total_load_balance = Constraint(T, rule=total_load_balance_rule)

    # Energy balance
    E_d = (model.PD_arb  
           + beta['reg_up'] * model.PD_reg
           + beta['pres'] * model.P_pres
           + beta['cres'] * model.P_cres
           + beta['dr'] * model.P_dr
           + beta['il'] * model.P_il
           ) * dt
    
    E_dr = (
           +  (gamma['reg_up'] - beta['reg_up']) * model.PD_reg
           +  (gamma['pres'] - beta['pres']) * model.P_pres
           +  (gamma['cres'] - beta['cres']) * model.P_cres
           +  (gamma['dr'] - beta['dr']) * model.P_dr
           +  (gamma['il'] - beta['il']) * model.P_il
           ) * dt

    E_c = (model.PC_arb  
           + beta['reg_down'] * model.PC_reg
           + model.PC_ec  
           ) * dt

    E_cr = ((gamma['reg_down'] - beta['reg_down']) * model.PC_reg) * dt

    def soc_constraints(model, t):
        if t == 0:
            return model.SOC[t] == initial_soc
        else:
            return model.SOC[t] == model.SOC[t-1] * Seff + E_c[t] * Ceff - E_d[t]/Deff
    model.SOC_constraints = Constraint(SOC_T, rule=soc_constraints)

    def soc_lower_limit_constraint(model, t):
        return model.SOC[t] - E_dr[t]/Deff >= cap_energy * λ_min[t]
    model.SOC_lower_limit = Constraint(T, rule=soc_lower_limit_constraint)

    def soc_upper_limit_constraint(model, t):
        return model.SOC[t] + E_cr[t] * Ceff <= cap_energy * λ_max[t]
    model.SOC_upper_limit = Constraint(T, rule=soc_upper_limit_constraint)

    if current_period == last_period:
        def soc_final_state_constraint(model):
            end_t = SOC_T[-1]
            return model.SOC[end_t] == start_soc 
        model.SOC_final = Constraint(rule=soc_final_state_constraint)


    # Cycle limits

    if z['activate'] == 1:

        def cycle_charge_constraint(model):
            return sum(model.PC_arb[t] +  model.PC_reg[t] * gamma['reg_down'] for t in T)*dt <= η * cap_energy * (λ_max_tech - λ_min_tech)
        model.cycle_charge = Constraint(rule=cycle_charge_constraint)

        def cycle_discharge_constraint(model):
            return sum(model.PD_arb[t] + model.PD_reg[t] * gamma['reg_up'] for t in T)*dt <= η * cap_energy * (λ_max_tech - λ_min_tech)
        model.cycle_discharge = Constraint(rule=cycle_discharge_constraint)

    else:

        def cycle_charge_constraint(model):
            return sum(model.PC_arb[t] +  model.PC_reg[t] * beta['reg_down'] for t in T)*dt <= η * cap_energy * (λ_max_tech - λ_min_tech)
        model.cycle_charge = Constraint(rule=cycle_charge_constraint)

        def cycle_discharge_constraint(model):
            return sum(model.PD_arb[t] + model.PD_reg[t] * beta['reg_up'] for t in T)*dt <= η * cap_energy * (λ_max_tech - λ_min_tech)
        model.cycle_discharge = Constraint(rule=cycle_discharge_constraint)


    # Service availability

    def arbitrage_availability_rule(model, t):
        return model.PC_arb[t] + model.PD_arb[t] <= cap_power * z['arb'] * theta['arb'].loc[t] * 2
    model.arbitrage_availability = Constraint(T, rule=arbitrage_availability_rule)

    def regulation_availability_rule(model, t):
        return model.PC_reg[t] + model.PD_reg[t] <= cap_power * z['reg'] * theta['reg'].loc[t] * 2
    model.regulation_availability = Constraint(T, rule=regulation_availability_rule)

    def preserve_availability_rule(model, t):
        return model.P_pres[t] <= cap_power * z['pres'] * theta['pres'].loc[t] * 2
    model.preserve_availability = Constraint(T, rule=preserve_availability_rule)

    def creserve_availability_rule(model, t):
        return model.P_cres[t] <= cap_power * z['cres'] * theta['cres'].loc[t] * 2
    model.creserve_availability = Constraint(T, rule=creserve_availability_rule)

    def energy_charge_availability_rule(model, t):
        return model.PC_ec[t] <= cap_power * z['ec'] * theta['ec'].loc[t] * 2
    model.energy_charge_availability = Constraint(T, rule=energy_charge_availability_rule)

    def demand_response_availability_rule(model, t):
        return model.P_dr[t] <= cap_power * z['dr'] * theta['dr'].loc[t] * 2
    model.demand_response_availability = Constraint(T, rule=demand_response_availability_rule)

    def interruptible_load_availability_rule(model, t):
        return model.P_il[t] <= cap_power * z['il'] * theta['il'].loc[t] * 2
    model.interruptible_load_availability = Constraint(T, rule=interruptible_load_availability_rule)


    # Solve the model
    solver = SolverFactory('gurobi_direct')
    solver.solve(model, tee=True)

    # Extract results for charging, discharging, and SOC
    charging_schedule = [model.PC_arb[t].value for t in T]  # Charging as negative
    discharging_schedule = [model.PD_arb[t].value for t in T]  # Discharging as positive
    reg_down_schedule = [model.PC_reg[t].value for t in T]  # Charging as negative
    reg_up_schedule = [model.PD_reg[t].value for t in T]  # Discharging as positive
    pres_schedule = [model.P_pres[t].value for t in T]  # Charging as negative
    cres_schedule = [model.P_cres[t].value for t in T]  # Discharging as positive

    soc_schedule = [model.SOC[t].value for t in SOC_T]  # State of Charge

    rev_arbitrage =  sum(model.R_arb[t] for t in T)() 
    rev_regulation = sum(model.R_reg[t] for t in T)() 
    rev_prim_reserve = sum(model.R_pres[t] for t in T)() 
    rev_cont_reserve = sum(model.R_cres[t] for t in T)() 

    # Return the results and final SOC
    return model.obj(), [charging_schedule, discharging_schedule, soc_schedule, reg_down_schedule, reg_up_schedule, pres_schedule, cres_schedule], [rev_arbitrage, rev_regulation, rev_prim_reserve, rev_cont_reserve]

# Run

In [ ]:
# Lists to store results
total_revenue = 0
arbitrage = 0
regulation = 0
preserve = 0
creserve = 0

all_charging_schedules = []
all_discharging_schedules = []
all_reg_down_schedules = []
all_reg_up_schedules = []
all_pres_schedules = []
all_cres_schedules = []
all_soc_schedules = []

first_p = 0
last_p = num_slices #- 1

# Run the optimization for each time period
for p in range(num_slices):
    print("watch", p)
    periodic_price = price_df[p*model_time_period:(p+1)*model_time_period]
    revenue, [charging_schedule, discharging_schedule, soc_schedule, reg_down_schedule, reg_up_schedule, pres_schedule, cres_schedule], [rev_arbitrage, rev_regulation, rev_prim_reserve, rev_cont_reserve] = optimize_revenue(initial_soc, periodic_price, p, first_p, last_p)
    final_soc = soc_schedule[-1]
    
    # Store the results
    total_revenue += revenue
    arbitrage += rev_arbitrage
    regulation += rev_regulation
    preserve += rev_prim_reserve
    creserve += rev_cont_reserve

    all_charging_schedules.extend(charging_schedule)
    all_discharging_schedules.extend(discharging_schedule)
    all_reg_down_schedules.extend(reg_down_schedule)
    all_reg_up_schedules.extend(reg_up_schedule)
    all_pres_schedules.extend(pres_schedule)
    all_cres_schedules.extend(cres_schedule)
    all_soc_schedules.extend(soc_schedule[:-1])

    # Update the initial SOC for the next day
    initial_soc = final_soc

print(total_revenue)

In [ ]:
revenue_data = pd.Series()
revenue_data.loc['Arbitrage'] = arbitrage/num_slices * 365
revenue_data.loc['Regulation'] = regulation/num_slices * 365
revenue_data.loc['Primary Reserve'] = preserve/num_slices * 365
revenue_data.loc['Contingency Reserve'] = creserve/num_slices * 365

pd.DataFrame(revenue_data, columns = ['revenue'])

In [ ]:
df_filtered = revenue_data[revenue_data.values > 0]

fig = go.Figure(data=[go.Pie(labels=df_filtered.index, values=df_filtered.values, hole=0.3)])

# Update layout for the pie chart
fig.update_layout(
    title_text='Breakdown of Annual Revenues',
    annotations=[dict(text=' ', x=0.5, y=0.5, font_size=20, showarrow=False)]
)

# Display the plot in a notebook or interactive environment
fig.show()

In [ ]:
# Create a DataFrame for the results
data = {
    'time': price_df.index,
    'charge': all_charging_schedules,
    'discharge': all_discharging_schedules,
    'reg_down': all_reg_down_schedules,
    'reg_up': all_reg_up_schedules,
    'pres': all_pres_schedules,
    'cres': all_cres_schedules,
    'soc': all_soc_schedules
}

result_df = pd.DataFrame(data)

#result_df = result_df.set_index(['t'])

result_df['net_discharge'] = result_df['discharge'] - result_df['charge']
result_df['soc_percent'] = result_df['soc'] /cap_energy

result_df

In [ ]:
price_df.index.names = ['time']
price_data_ex = price_df.reset_index().to_json(orient='records', date_format='iso')
result_data_ex = result_df.to_json(orient='records', date_format='iso')

In [ ]:
from scripts.result_export import generate_html

BESS_icost = 207
BESS_ecost = 355
BESS_pcost = 153

annual_revenue = total_revenue/num_slices * 365
discount_rate = 0.05
om_percentage = 0.05

generate_html(filename, BESS_icost, BESS_ecost, BESS_pcost, cap_energy, cap_power, annual_revenue, price_data_ex, result_data_ex, revenue_data, discount_rate, om_percentage, calendar_life)

In [ ]:
initial_cost

In [ ]:
BESS_icost = 207000
BESS_ecost = 355000
BESS_pcost = 153000

initial_cost = 207000 + 355000 * cap_energy + 153000*cap_power
annual_cost = initial_cost * 0.05
annual_revenue = total_revenue/num_slices * 365
discount_rate = 0.05
om_percentage = 0.05

In [ ]:
initial_cost

In [ ]:
annual_cost

# Plot

In [ ]:
df = result_df
df = df.set_index(['time'])

#df['d'] = df['time'].dt.date
#df['t'] = df['time'].dt.time
#df = df.set_index(['d', 't']).drop(columns = ['time'])


In [ ]:
start_time = 0
end_time = 48

# Create price figure
fig_price = go.Figure()
fig_price.add_trace(go.Scatter(x=price_df[start_time:end_time].index, y=price_df['arb_energy_price'], mode='lines', name='Energy'))
fig_price.add_trace(go.Scatter(x=price_df[start_time:end_time].index, y=price_df['reg_up_price'], mode='lines', name='Regulation Up'))
fig_price.add_trace(go.Scatter(x=price_df[start_time:end_time].index, y=price_df['reg_down_price'], mode='lines', name='Regulation Down'))
fig_price.add_trace(go.Scatter(x=price_df[start_time:end_time].index, y=price_df['pres_capacity_price'], mode='lines', name='Primary Reserve'))
fig_price.add_trace(go.Scatter(x=price_df[start_time:end_time].index, y=price_df['cres_capacity_price'], mode='lines', name='Contingency Reserve'))

fig_price.update_layout(
    title='Prices',
    xaxis_title='Time',
    yaxis_title='$/MWh',
    legend=dict(
        orientation='h',
        x=0,
        y=1.1
    ),
    autosize=True,
    margin=dict(l=40, r=40, t=80, b=40),
)

# Create charging and discharging figure
fig_charge_discharge = go.Figure()
fig_charge_discharge.add_trace(go.Scatter(x=result_df[start_time:end_time].index, y=result_df['net_discharge'], mode='lines', name='Net Power Discharge'))
fig_charge_discharge.add_trace(go.Scatter(x=result_df[start_time:end_time].index, y=result_df['reg_up'], mode='lines', name='Regulation Up'))
fig_charge_discharge.add_trace(go.Scatter(x=result_df[start_time:end_time].index, y=result_df['reg_down'], mode='lines', name='Regulation Down'))
fig_charge_discharge.add_trace(go.Scatter(x=result_df[start_time:end_time].index, y=result_df['pres'], mode='lines', name='Primary Reserve'))
fig_charge_discharge.add_trace(go.Scatter(x=result_df[start_time:end_time].index, y=result_df['cres'], mode='lines', name='Contingency Reserve'))

fig_charge_discharge.update_layout(
    title='Charging and Discharging Schedule',
    xaxis_title='Time',
    yaxis_title='Power (MW)',
    legend=dict(
        orientation='h',
        x=0,
        y=1.1
    ),
    autosize=True,
    margin=dict(l=40, r=40, t=80, b=40),
)

# Create state of charge figure
fig_soc = go.Figure()
fig_soc.add_trace(go.Scatter(x=result_df.index[start_time:end_time], y=result_df['soc_percent']*100, mode='lines', name='net', line=dict(color='orange')))
fig_soc.update_layout(
    title='State of Charge',
    xaxis_title='Time',
    yaxis_title='SOC (%)',
    legend=dict(
        orientation='h',
        x=0,
        y=1.1
    ),
    autosize=True,
    margin=dict(l=40, r=40, t=80, b=40),
)

# Generate HTML snippets for each figure
price_html = pio.to_html(fig_price, full_html=False)
charge_discharge_html = pio.to_html(fig_charge_discharge, full_html=False)
soc_html = pio.to_html(fig_soc, full_html=False)

# Combine all HTML snippets into a single HTML document
full_html = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Figures</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
</head>
<body>
    <h1>Price Figure</h1>
    {price_html}
    <h1>Charging and Discharging Schedule</h1>
    {charge_discharge_html}
    <h1>State of Charge</h1>
    {soc_html}
</body>
</html>
"""

# Save to an HTML file
with open("result/day/" + filename + ".html", "w",  encoding="utf-8") as file:
    file.write(full_html)


